In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

import pinecone
import langchain


c:\Users\anike\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
pinecone_api_key= "56bd8024-e9fa-4763-af69-1d32202e7e83"

In [4]:
pinecone_api_env = "gcp-starter"

In [5]:
#extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("data/")

In [ ]:
#extracted_data

In [ ]:
# create chunks
def text_split(extracted_data):
    chunks = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap =20)
    text_chunks = chunks.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)

In [ ]:
print("len of my chunks:", len(text_chunks))

len of my chunks: 7020


In [ ]:
# download embeding model
def download_embedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [ ]:
embedding = download_embedding()

In [ ]:
query_result = embedding.embed_query("hello world")
print(len(query_result))

384


In [ ]:
# initialize pinecone
pc = pinecone.Pinecone(api_key = pinecone_api_key)
pc.list_indexes()


{'indexes': [{'dimension': 384,
              'host': 'chatbot-3pgpjm9.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'chatbot',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [ ]:
name = 'chatbot'
index = pc.Index(name)

In [ ]:
# embed each chunk and upsert it to index
for i,t in zip(range(len(text_chunks)),text_chunks):
    query = embedding.embed_query(t.page_content)
    index.upsert(
        vectors=[
            {
                "id":str(i),
                "values":query,
                "metadata":{"text":str(text_chunks[i].page_content)}
            }
        ],

        namespace="real"
    )

In [ ]:
#search = Pinecone.from_existing_index(name,embedding)
query1 =embedding.embed_query( "what are allergies")
data = index.query(namespace="real",
            vector=query1,
            top_k=3,
            include_metadata=True,
            include_values=False,
            filter = {}
            )


In [ ]:
texts = [match['metadata']['text'] for match in data['matches']]
query_text = "".join(texts)

NameError: name 'data' is not defined

In [50]:
# the answer generated is not readabel so we use llm to finetune the answer
prompt_template = """

use the following pieces of information to answe the user's questions.
if you dont know the answer just say that you dont know.

context = {context}
question = {question}

only return th ehelpful answer below and nothing else.
Helpful answer:
"""

In [51]:
prompt = PromptTemplate(template=prompt_template,input_variables=['context','question'])
chain_type_kwargs = {'prompt':prompt}

In [52]:
llm = CTransformers(model = "model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config = {'max_new_tokens':512,
                              'temperature':0.8})

In [64]:
p = prompt.format(context =query_text, question = "what are allergies")

In [65]:
tuned_response = llm(p)